In [1]:
from  osgeo import ogr, osr
import psycopg2
import shapely
import shapely.wkt
import geopandas as gpd
import math
%matplotlib inline

In [2]:
def interesting_alpha(tablename):
    table = get_data(tablename)
    sum = 0
    alpha = {}
    for val in table:
        sum += val[9]
    
    for val in table:
        alpha.update({val[0]:(val[9]/sum)})
    return(alpha)

In [3]:
def interesting_beta(tablename):
    #use for growth/shrinks
    table = get_data(tablename)
    beta = {}
    for val in table:
        beta.update({val[0]:float(val[8])})
    return (beta)

In [4]:
def interesting_gamma(tablename):
    #use for shifts
    table = get_data(tablename)
    
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('SELECT ST_Distance_Sphere(geometry(a.bot), geometry(b.top))'
                   'FROM '+
                    ' (SELECT st_astext(st_makepoint(st_xmin(st_extent(j2s.geom)), st_ymin(st_extent(j2s.geom)))) AS bot FROM '+tablename+' j2s) a, '
                    ' (SELECT st_astext(st_makepoint(st_xmax(st_extent(j2s.geom)), st_ymax(st_extent(j2s.geom)))) AS top FROM '+tablename+' j2s) b')
    distance = cursor.fetchall()
    connection.commit()
    
    gamma ={}
    for val in table:
        gamma.update({val[0]:int(val[8] >= (distance[0][0]/2))})
        
    return (gamma)

In [5]:
def interesting_delta(tablename):
    #Thinness ratio
    table = get_data(tablename)
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('SELECT ST_Perimeter(j2s.geom, true)'
                   'FROM '+tablename+' j2s ')
    perimeter = cursor.fetchall()
    connection.commit()

    thin_ratio = 0
    delta = {}

    for p, a in zip(perimeter, table):
        thin_ratio = (4 * math.pi * a[9]) / (p[0] ** 2)
        delta.update({a[0]:thin_ratio})

    return (delta)

In [6]:
def interesting_omega(alpha, beta, delta):
    omega = {}

    for val in range(1, (len(alpha)+1)):
        omega.update({val: (0.3 * alpha[val]) + (0.4 * beta[val]) + (0.4 * delta[val])})

    return (omega)

In [7]:
def omegaToTable(omega, tablename):
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('ALTER TABLE '+tablename+' ADD COLUMN IF NOT EXISTS interestingness numeric')
    for elem in omega:
        cursor.execute('UPDATE '+tablename+' SET interestingness= %s WHERE gid = %s',(omega[elem], elem))
    connection.commit()
    

In [8]:
def addAreaCol(tablename):
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('ALTER TABLE '+tablename+' ADD COLUMN IF NOT EXISTS area double precision')
    cursor.execute('UPDATE '+tablename+' SET area=ROUND((ST_Area(geom::geography))::numeric,2)')
    connection.commit()

In [9]:
def get_data(tableName):
    connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
    cursor = connection.cursor()
    cursor.execute('SELECT j2s.* '+
                'FROM '+tableName+' j2s ')
    temp_table = cursor.fetchall()
    connection.commit()
    return (temp_table)

In [10]:
#Testing for Interesting-ness parameters for the Change Parameters on June 3 and June 4
#First, Shrink
table = 'public.june3june4shrink'
addAreaCol(table)
a34 = interesting_alpha(table)
b34 = interesting_beta(table)
d34 = interesting_delta(table)
omegaToTable(interesting_omega(a34, b34, d34), table)

In [11]:
#Second, Growth
table = 'public.june3june4growth'
addAreaCol(table)
a34 = interesting_alpha(table)
b34 = interesting_beta(table)
d34 = interesting_delta(table)
omegaToTable(interesting_omega(a34, b34, d34), table)

In [12]:
#Third, Shift
table = 'public.june3june4shift'
addAreaCol(table)
a34 = interesting_alpha(table)
c34 = interesting_gamma(table)
d34 = interesting_delta(table)
omegaToTable(interesting_omega(a34, c34, d34), table)

In [13]:
#Now running this for all dates
#Comparing all sequential days
polygons =[]
for val in range(1,7):
    temp = 'public.june'+str(val)+'june'+str(val+1)
    polygons.append([temp+'shrink',temp+'growth',temp+'shift'])

for elem in polygons:
    #Shrink
    addAreaCol(elem[0])
    a34 = interesting_alpha(elem[0])
    b34 = interesting_beta(elem[0])
    d34 = interesting_delta(elem[0])
    omegaToTable(interesting_omega(a34, b34, d34), elem[0])
    
    #Growth
    addAreaCol(elem[1])
    a34 = interesting_alpha(elem[1])
    b34 = interesting_beta(elem[1])
    d34 = interesting_delta(elem[1])
    omegaToTable(interesting_omega(a34, b34, d34), elem[1])
    
    #Shift
    addAreaCol(elem[2])
    a34 = interesting_alpha(elem[2])
    c34 = interesting_gamma(elem[2])
    d34 = interesting_delta(elem[2])
    omegaToTable(interesting_omega(a34, c34, d34), elem[2])
    

In [111]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT j2s.* '+
                'FROM public.june12shrink j2s ')
shrink_table = cursor.fetchall()
connection.commit()

In [112]:
for val in shrink_table:
    print(val[8])

0.00192899379032392
0.00407155921941685
0.0317208550095566
0.00750897043828223
0.00762517440995081
0.197109198755777
0.0155506870186542


In [74]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT st_asText(ST_Extent(j2s.geom))'+
                'FROM public.june12shift j2s ')
bounding_box = cursor.fetchall()
connection.commit()

In [83]:
for val in bounding_box:
    print(val[0])

POLYGON((-78.97815059 42.00177517,-78.97815059 42.7230198,-76.53217853 42.7230198,-76.53217853 42.00177517,-78.97815059 42.00177517))


In [84]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('select st_astext(st_makepoint(st_xmin(st_extent(j2s.geom)), st_ymin(st_extent(j2s.geom)))) '+
                'FROM public.june12shift j2s ')
bot_left = cursor.fetchall()
connection.commit()

In [85]:
for val in bot_left:
    print(val[0])

POINT(-78.97815059 42.00177517)


In [86]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('select st_astext(st_makepoint(st_xmax(st_extent(j2s.geom)), st_ymax(st_extent(j2s.geom)))) '+
                'FROM public.june12shift j2s ')
top_right = cursor.fetchall()
connection.commit()

In [88]:
for val in top_right:
    print(val[0])

POINT(-76.53217853 42.7230198)


In [94]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT ST_Distance_Sphere(geometry(a.bot), geometry(b.top))'
                   'FROM '+
                    ' (SELECT st_astext(st_makepoint(st_xmin(st_extent(j2s.geom)), st_ymin(st_extent(j2s.geom)))) AS bot FROM public.june12shift j2s) a, '
                    ' (SELECT st_astext(st_makepoint(st_xmax(st_extent(j2s.geom)), st_ymax(st_extent(j2s.geom)))) AS top FROM public.june12shift j2s) b')
distance = cursor.fetchall()
connection.commit()

In [98]:
print(distance[0][0])

216366.16448114


In [99]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT j2s.* '+
                'FROM public.june12shift j2s ')
shift_table = cursor.fetchall()
connection.commit()

In [103]:
gamma ={}
for val in shift_table:
    gamma.update({val[0]:int(val[8] >= (distance[0][0]/2))})
print(gamma)

{1: 0, 2: 1, 3: 0}


In [76]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('ALTER TABLE public.june12shift ADD COLUMN IF NOT EXISTS area double precision')
cursor.execute('UPDATE public.june12shift SET area=ROUND((ST_Area(geom::geography))::numeric,2)')
cursor.execute('SELECT * FROM public.june12shift j2s ')
shift_data = cursor.fetchall()
connection.commit()

In [110]:
sum = 0
alpha = {}
for val in shift_table:
    sum += val[9]
    
for val in shift_table:
    alpha.update({val[0]:(val[9]/sum)})
print(alpha)

{1: 0.07249261563658106, 2: 0.09656910071873116, 3: 0.8309382836446878}


In [117]:
beta = {}
for val in shrink_table:
    beta.update({val[0]:float(val[8])})
print (beta)

{1: 0.00192899379032392, 2: 0.00407155921941685, 3: 0.0317208550095566, 4: 0.00750897043828223, 5: 0.00762517440995081, 6: 0.197109198755777, 7: 0.0155506870186542}


In [79]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT ' +
                'ST_Length(ST_LongestLine(' +
                    '(SELECT (ST_Extent(j2s.geom)) FROM public.june12shift j2s), ' +
                    '(SELECT (ST_Extent(j2s.geom)) FROM public.june12shift j2s)))')
longest_line = cursor.fetchall()

In [80]:
for val in longest_line:
    print(val[0])

2.55009276980358


In [68]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT (ST_MaxDistance(' +
                    '(SELECT (ST_Extent(j2s.geom)) FROM public.june12shift j2s), ' +
                    '(SELECT (ST_Extent(j2s.geom)) FROM public.june12shift j2s)))')
max_line = cursor.fetchall()
connection.commit()

In [69]:
for val in max_line:
    print(val[0])

2.55009276980358


In [121]:
#Thinness ratio
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('SELECT ST_Perimeter(j2s.geom, true)'
                   'FROM public.june12shift j2s ')
perimeter = cursor.fetchall()
connection.commit()

In [133]:
thin_ratio = 0
delta = {}

for p, a in zip(perimeter, shift_table):
    thin_ratio = (4 * math.pi * a[9]) / (p[0] ** 2)
    delta.update({a[0]:thin_ratio})

print (delta)

{1: 0.7083695135291909, 2: 0.7582329274298542, 3: 0.35376134899877115}


In [140]:
omega = {}

for val in range(1, (len(shift_table)+1)):
    #pass
    omega.update({val: (0.3 * alpha[val]) + (0.4 * beta[val]) + (0.4 * gamma[val])})

print (omega)


{1: 0.022519382207103887, 2: 0.4305993539033861, 3: 0.261969827097229}


In [178]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('ALTER TABLE public.june12shrink ADD COLUMN IF NOT EXISTS area double precision')
cursor.execute('UPDATE public.june12shrink SET area=ROUND((ST_Area(geom::geography))::numeric,2)')
cursor.execute('SELECT * FROM public.june12shrink j2s ')
shrink_table = cursor.fetchall()
connection.commit()

In [179]:
a = interesting_alpha(shrink_table)
b = interesting_beta(shrink_table)
d = interesting_delta(shrink_table, 'public.june12shrink')

In [180]:
o = interesting_omega(a,b,d)

In [195]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('ALTER TABLE public.june12shrink ADD COLUMN IF NOT EXISTS interestingness numeric')
for elem in o:
    cursor.execute('UPDATE public.june12shrink SET interestingness= %s WHERE gid = %s',(o[elem], elem))
connection.commit()

In [198]:
a_shift = interesting_alpha(shift_table)
c_shift = interesting_gamma(shift_table, 'public.june12shift')
d_shift = interesting_delta(shift_table, 'public.june12shift')
o_shift = interesting_omega(a_shift,c_shift,d_shift)

[(216366.16448114,)]


In [199]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('ALTER TABLE public.june12shift ADD COLUMN IF NOT EXISTS interestingness numeric')
for elem in o_shift:
    cursor.execute('UPDATE public.june12shift SET interestingness= %s WHERE gid = %s',(o_shift[elem], elem))
connection.commit()

In [200]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('ALTER TABLE public.june12growth ADD COLUMN IF NOT EXISTS area double precision')
cursor.execute('UPDATE public.june12growth SET area=ROUND((ST_Area(geom::geography))::numeric,2)')
cursor.execute('SELECT * FROM public.june12growth j2s ')
growth_table = cursor.fetchall()
connection.commit()

In [203]:
a_growth = interesting_alpha(growth_table)
b_growth = interesting_beta(growth_table)
d_growth = interesting_delta(growth_table, 'public.june12growth')
o_growth = interesting_omega(a_growth,b_growth,d_growth)

In [204]:
connection = psycopg2.connect(database="twitter_change",user="postgres", password="password")
cursor = connection.cursor()
cursor.execute('ALTER TABLE public.june12growth ADD COLUMN IF NOT EXISTS interestingness numeric')
for elem in o_growth:
    cursor.execute('UPDATE public.june12growth SET interestingness= %s WHERE gid = %s',(o_growth[elem], elem))
connection.commit()